In [1]:
import pyarrow as pa
import pyarrow.flight

In [2]:
client = pa.flight.connect('grpc://0.0.0.0:8815')

In [5]:
# List existing datasets.
flights = [flight for flight in client.list_flights()]
# flights = [flight for flight in client.list_flights()]
row_count=0
for flight in flights:
    descriptor = flight.descriptor
    row_count+=flight.total_records
    print("Path:", descriptor.path[0].decode('utf-8'), "Rows:", flight.total_records, "Size:", flight.total_bytes)

Path: right_image_20220404150457.parquet Rows: 257 Size: 1226
Path: ubx_nav_hp_pos_llh_20220404150357.parquet Rows: 110 Size: 2472
Path: vehicle_state_20220404150457.parquet Rows: 979 Size: 2277
Path: point_cloud2_20220404150457.parquet Rows: 205 Size: 1732
Path: point_cloud2_20220404150357.parquet Rows: 241 Size: 1732
Path: left_image_20220404133715.parquet Rows: 184 Size: 1226
Path: left_image_20220404145832.parquet Rows: 1641 Size: 2183
Path: ubx_nav_hp_pos_llh_20220404150457.parquet Rows: 102 Size: 2470
Path: vehicle_state_20220404150357.parquet Rows: 667 Size: 2270
Path: left_image_20220404142846.parquet Rows: 2984 Size: 3117
Path: right_image_20220404150357.parquet Rows: 321 Size: 1226
Path: left_image_20220404145108.parquet Rows: 459 Size: 1226
Path: left_image_20220404143213.parquet Rows: 249 Size: 1226
Path: right_image_20220404133938.parquet Rows: 158 Size: 1222
Path: left_image_20220404140118.parquet Rows: 920 Size: 1253
Path: point_cloud2_20220404133938.parquet Rows: 117 Si

In [6]:
# List existing datasets.
flights = [flight for flight in client.list_flights(criteria=bytes('**/point_cloud2*.parquet','utf-8'))]
# flights = [flight for flight in client.list_flights()]
row_count=0
for flight in flights:
    descriptor = flight.descriptor
    row_count+=flight.total_records
    print("Path:", descriptor.path[0].decode('utf-8'), "Rows:", flight.total_records, "Size:", flight.total_bytes)
    # print("=== Schema ===")
    # print(flight.schema)
    # print("==============")
    # print("")
    
print("row_count:" , row_count)

Path: point_cloud2_20220404150457.parquet Rows: 205 Size: 1732
Path: point_cloud2_20220404150357.parquet Rows: 241 Size: 1732
Path: point_cloud2_20220404133938.parquet Rows: 117 Size: 1721
Path: point_cloud2_20220404143213.parquet Rows: 178 Size: 1732
Path: point_cloud2_20220404140118.parquet Rows: 668 Size: 1764
Path: point_cloud2_20220404145108.parquet Rows: 332 Size: 1734
Path: point_cloud2_20220404133715.parquet Rows: 142 Size: 1728
Path: point_cloud2_20220404145832.parquet Rows: 1289 Size: 3143
Path: point_cloud2_20220404142846.parquet Rows: 2390 Size: 4515
row_count: 5562


In [7]:
f1_descriptor=pa.flight.FlightDescriptor.for_path("ubx_nav_hp_pos_llh_20220404150357.parquet")

In [8]:
f1 = client.get_flight_info(f1_descriptor)

In [9]:
print("Path:", f1_descriptor.path[0].decode('utf-8'), "Rows:", f1.total_records, "Size:", f1.total_bytes)
print("=== Schema ===")
print(f1.schema)
print("==============")

Path: ubx_nav_hp_pos_llh_20220404150357.parquet Rows: 110 Size: 2472
=== Schema ===
timestamp: timestamp[ns, tz=UTC] not null
header: struct<stamp: struct<sec: int32, nanosec: uint32>, frame_id: string> not null
  child 0, stamp: struct<sec: int32, nanosec: uint32>
      child 0, sec: int32
      child 1, nanosec: uint32
  child 1, frame_id: string
version: uint8 not null
invalid_lon: bool not null
invalid_lat: bool not null
invalid_height: bool not null
invalid_hmsl: bool not null
invalid_lon_lp: bool not null
invalid_lat_hp: bool not null
invalid_height_hp: bool not null
invalid_hmsl_hp: bool not null
itow: uint32 not null
lon: int32 not null
lat: int32 not null
height: int32 not null
hmsl: int32 not null
lon_hp: int8 not null
lat_hp: int8 not null
height_hp: int8 not null
hmsl_hp: int8 not null
h_acc: uint32 not null
v_acc: uint32 not null


In [10]:
reader = client.do_get(f1.endpoints[0].ticket)
read_table = reader.read_all()
read_table.to_pandas().head()

,timestamp,header,version,invalid_lon,invalid_lat,invalid_height,invalid_hmsl,invalid_lon_lp,invalid_lat_hp,invalid_height_hp,...,lon,lat,height,hmsl,lon_hp,lat_hp,height_hp,hmsl_hp,h_acc,v_acc
0,2021-12-14 04:44:32.729063424+00:00,"{'stamp': {'sec': 1639457072, 'nanosec': 72906...",0,False,False,False,False,False,False,False,...,1534214926,-280000147,41129,3855,13,3,-2,1,8974,12950
1,2021-12-14 04:44:32.827851264+00:00,"{'stamp': {'sec': 1639457072, 'nanosec': 82785...",0,False,False,False,False,False,False,False,...,1534214927,-280000147,41133,3859,30,49,-2,1,8971,12947
2,2021-12-14 04:44:33.028940288+00:00,"{'stamp': {'sec': 1639457073, 'nanosec': 28940...",0,False,False,False,False,False,False,False,...,1534214926,-280000146,41125,3851,-18,-34,2,5,8966,12940
3,2021-12-14 04:44:33.134975744+00:00,"{'stamp': {'sec': 1639457073, 'nanosec': 13497...",0,False,False,False,False,False,False,False,...,1534214925,-280000145,41118,3844,16,26,-5,-2,8964,12937
4,2021-12-14 04:44:33.230634752+00:00,"{'stamp': {'sec': 1639457073, 'nanosec': 23063...",0,False,False,False,False,False,False,False,...,1534214923,-280000143,41149,3875,48,45,-4,0,8962,12934


In [11]:
read_table.num_rows

110

In [12]:
# List existing datasets.
flights = [flight for flight in client.list_flights(criteria=bytes('point_cloud2.dataset','utf-8'))]
print(flights)
# flights = [flight for flight in client.list_flights()]
for flight in flights:
    descriptor = flight.descriptor
    print("Path:", descriptor.path[0].decode('utf-8'), "Rows:", flight.total_records, "Size:", flight.total_bytes)

Path: point_cloud2.dataset Rows: 5562 Size: -1


In [13]:
f2_descriptor=pa.flight.FlightDescriptor.for_path("point_cloud2.dataset")

In [14]:
f2 = client.get_flight_info(f2_descriptor)

In [15]:
print("Path:", f2_descriptor.path[0].decode('utf-8'), "Rows:", f2.total_records, "Size:", f2.total_bytes)
print("=== Schema ===")
print(f2.schema)
print("==============")

Path: point_cloud2.dataset Rows: 5562 Size: -1
=== Schema ===
timestamp: timestamp[ns, tz=UTC] not null
header: struct<stamp: struct<sec: int32, nanosec: uint32>, frame_id: string>
  child 0, stamp: struct<sec: int32, nanosec: uint32>
      child 0, sec: int32
      child 1, nanosec: uint32
  child 1, frame_id: string
height: uint32
width: uint32
fields: list<element: struct<name: string, offset: uint32, datatype: uint8, count: uint32>>
  child 0, element: struct<name: string, offset: uint32, datatype: uint8, count: uint32>
      child 0, name: string
      child 1, offset: uint32
      child 2, datatype: uint8
      child 3, count: uint32
is_bigendian: bool
point_step: uint32
row_step: uint32
data: binary
is_dense: bool


In [16]:
reader = client.do_get(f2.endpoints[0].ticket)
read_table = reader.read_all()
read_table.to_pandas().head()

,timestamp,header,height,width,fields,is_bigendian,point_step,row_step,data,is_dense
0,2021-12-14 04:43:47.015560448+00:00,"{'stamp': {'sec': 1639457027, 'nanosec': 15560...",16,2232,"[{'name': 'x', 'offset': 0, 'datatype': 7, 'co...",False,32,71424,b'\r\xfb\xb3@\xef\xb0+\xbf\x19:\xc1\xbf\x00\x0...,False
1,2021-12-14 04:43:47.075813632+00:00,"{'stamp': {'sec': 1639457027, 'nanosec': 75813...",16,2232,"[{'name': 'x', 'offset': 0, 'datatype': 7, 'co...",False,32,71424,b't<\xb4@\x9e\x95\x19\xbf\x19:\xc1\xbf\x00\x00...,False
2,2021-12-14 04:43:47.135667456+00:00,"{'stamp': {'sec': 1639457027, 'nanosec': 13566...",16,2232,"[{'name': 'x', 'offset': 0, 'datatype': 7, 'co...",False,32,71424,b'\ru\xb4@\x1a\xf5\x07\xbf\x19:\xc1\xbf\x00\x0...,False
3,2021-12-14 04:43:47.165630464+00:00,"{'stamp': {'sec': 1639457027, 'nanosec': 16563...",16,2232,"[{'name': 'x', 'offset': 0, 'datatype': 7, 'co...",False,32,71424,b'0\xce\xb4@l\xd2\xec\xbe\x8fd\xc1\xbf\x00\x00...,False
4,2021-12-14 04:43:47.225682432+00:00,"{'stamp': {'sec': 1639457027, 'nanosec': 22568...",16,2256,"[{'name': 'x', 'offset': 0, 'datatype': 7, 'co...",False,32,72192,b'~ \xb5@/\xa2\xc9\xbe\x05\x8f\xc1\xbf\x00\x00...,False


In [17]:
read_table.num_rows

5562